In [1]:
! pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
import numpy as np
import pandas as pd

In [4]:
import bs4

In [5]:
import lxml.html as lh

In [6]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)

In [7]:
soup = bs4.BeautifulSoup(req.text, "html5lib")

In [8]:
data = soup.select('.wikitable.sortable')

In [9]:
print (type(data))
print (len(data))

<class 'list'>
1


In [10]:
doc = lh.fromstring(req.content)

In [11]:
tr_elements = doc.xpath('//tr')

In [12]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [13]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [14]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1


In [15]:
# Convert the list into dict and then into a dataframe
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [16]:
df = df[df.Borough != 'Not assigned']

In [17]:
df.rename(columns = {'Neighbourhood\n':'Neighbourhood'}, inplace = True)

In [18]:
for i in range(0,212):
    df.iloc[i,2] = df.iloc[i,2].strip('\n')

In [19]:
df = df.reset_index(drop = True)

In [20]:
for i in range(0,212):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]

In [21]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [22]:
df1 = (df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(set(x.dropna()))).reset_index())

In [23]:
df1.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park,East Birchmount Park,Ionview"
7,M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
8,M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [24]:
df1.shape

(103, 3)